# Kaggle Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install kaggle

In [ ]:
!mkdir .kaggle

In [ ]:
!ls

drive  sample_data


In [ ]:
import json
token = {'username':'username','key':'*************'}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [ ]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria -p /content

 99% 671M/675M [00:07<00:00, 130MB/s]
100% 675M/675M [00:07<00:00, 96.4MB/s]


In [ ]:
!unzip \*.zip

In [ ]:
import os
os.listdir('/content/cell_images/cell_images/Parasitized')[1]

'C140P101ThinF_IMG_20151005_211735_cell_150.png'

In [ ]:
os.listdir('/content/cell_images/Parasitized')[1]

'C140P101ThinF_IMG_20151005_211735_cell_150.png'

# Data setup

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
os.listdir('/content/cell_images/cell_images')

['Uninfected', 'Parasitized']

In [ ]:
pos_list = os.listdir('/content/cell_images/cell_images/Parasitized')

In [ ]:
neg_list = os.listdir('/content/cell_images/cell_images/Uninfected')

In [ ]:
print(len(pos_list))
print(len(neg_list))

13780
13780


In [ ]:
base = '/content/base'


In [ ]:
!mkdir '/content/base'

In [ ]:
train='/content/base/train'
os.mkdir(train)

In [ ]:
valid = '/content/base/valid'
os.mkdir(valid)

In [ ]:
train_pos = '/content/base/train/Parasitized'
train_neg  = '/content/base/train/Uninfected'
os.mkdir(train_pos)
os.mkdir(train_neg)

In [ ]:
valid_pos=('/content/base/valid/Parasitized')
valid_neg=('/content/base/valid/Uninfected')
os.mkdir(valid_pos)
os.mkdir(valid_neg)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
pos_train, pos_valid = train_test_split(pos_list,test_size=0.3)

In [ ]:
neg_train, neg_valid = train_test_split(neg_list,test_size=0.3)

In [ ]:
print(len(pos_train))
print(len(pos_valid))
print(len(neg_train))
print(len(neg_valid))

9646
4134
9646
4134


In [ ]:
from shutil import copyfile

In [ ]:
def label_data(train_list,valid_list,DEST_train,DEST_valid,name):
    for i in train_list:
        copyfile(os.path.join(name,i),DEST_train + '/' + i)
    
    for j in valid_list:
        copyfile(os.path.join(name,j),DEST_valid+ '/' + j)
    

In [ ]:
name1 = '/content/cell_images/cell_images/Parasitized'
name2  = '/content/cell_images/cell_images/Uninfected'

In [ ]:
pos = label_data(pos_train,pos_valid,train_pos,valid_pos,name1)

In [ ]:
neg = label_data(neg_train,neg_valid,train_neg,valid_neg,name2)

In [ ]:
os.listdir(valid_pos)[0:2]

['C101P62ThinF_IMG_20150918_151507_cell_51.png',
 'C91P52ThinF_IMG_20150821_124739_cell_193.png']

# TF Model 

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0,rotation_range=40,height_shift_range=0.2,
                                   width_shift_range=0.2,shear_range=0.2,zoom_range=0.2,
                                 horizontal_flip=True,fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
train_gen = train_datagen.flow_from_directory(train,target_size =(300,300),batch_size=64,class_mode='binary')
valid_gen = valid_datagen.flow_from_directory(valid,target_size =(300,300),batch_size=64,class_mode='binary')

Found 25874 images belonging to 2 classes.
Found 12095 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape=(300,300,3),include_top=False,weights='imagenet')

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output
last_layer.output_shape

(None, 17, 17, 768)

In [ ]:
# model creation
activation='relu'

model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Conv2D(64,(5,5),activation=activation,input_shape=(300,300,3)))
model1.add(tf.keras.layers.MaxPooling2D(2,2))
#2nd CNN
model1.add(tf.keras.layers.Conv2D(32,(5,5),activation=activation))
model1.add(tf.keras.layers.MaxPooling2D(2,2))
#3rd CNN
model1.add(tf.keras.layers.Conv2D(16,(5,5),activation=activation))
model1.add(tf.keras.layers.MaxPooling2D(2,2))
#4th CNN
model1.add(tf.keras.layers.Conv2D(8,(5,5),activation=activation))
model1.add(tf.keras.layers.MaxPooling2D(2,2))

model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(152,activation=activation))
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(152,activation=activation))
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(152,activation=activation))
model1.add(tf.keras.layers.BatchNormalization())

model1.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_188 (Conv2D)          (None, 296, 296, 64)      4864      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 148, 148, 64)      0         
_________________________________________________________________
conv2d_189 (Conv2D)          (None, 144, 144, 32)      51232     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 72, 72, 32)        0         
_________________________________________________________________
conv2d_190 (Conv2D)          (None, 68, 68, 16)        12816     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 34, 34, 16)        0         
_________________________________________________________________
conv2d_191 (Conv2D)          (None, 30, 30, 8)         3

In [ ]:
optimizer='adam'
loss = 'binary_crossentropy'
metrics=['accuracy']
model1.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [ ]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('accuracy')>0.92):
            print('Achieved 92% accuracy')
            self.model.stop_training=True
callback = mycallback()
    

In [ ]:
history = model1.fit(train_gen,epochs=20,verbose=1,validation_data=valid_gen,steps_per_epoch=80,callbacks=[callback],use_multiprocessing=False,workers=16)

Epoch 1/20
80/80 [==============================] - 116s 1s/step - loss: 0.4230 - accuracy: 0.8163 - val_loss: 0.3478 - val_accuracy: 0.8614
Epoch 2/20
80/80 [==============================] - 117s 1s/step - loss: 0.3545 - accuracy: 0.8555 - val_loss: 1.0784 - val_accuracy: 0.7773
Epoch 3/20
80/80 [==============================] - 117s 1s/step - loss: 0.2884 - accuracy: 0.8895 - val_loss: 0.1953 - val_accuracy: 0.9308
Epoch 4/20
80/80 [==============================] - 116s 1s/step - loss: 0.2828 - accuracy: 0.8995 - val_loss: 0.2631 - val_accuracy: 0.9172
Epoch 5/20
80/80 [==============================] - 117s 1s/step - loss: 0.2792 - accuracy: 0.9000 - val_loss: 0.6306 - val_accuracy: 0.5288
Epoch 6/20
80/80 [==============================] - 117s 1s/step - loss: 0.2808 - accuracy: 0.9054 - val_loss: 0.2416 - val_accuracy: 0.9252
Epoch 7/20
80/80 [==============================] - 117s 1s/step - loss: 0.2961 - accuracy: 0.8945 - val_loss: 0.3790 - val_accuracy: 0.8710
Epoch 8/20
80